In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data_path = "/Users/jahrios/Documents/Stanford/PoldrackLab/Projects/SharedControl/Data/raw/"
exp_stage = "test"
task = "simple_stop"

pattern = os.path.join(data_path, exp_stage, '*', task, '*.csv')

data_files = glob.glob(pattern)

In [4]:
def compute_SSRT(df, max_go_rt = 2, violation_flag = False):
    
    df = df.query('Phase == "test"')
    
    go_trials = df.loc[df.trialType == 'go']
    stop_df = df.loc[df.trialType == 'stop']

        
    go_replacement_df = go_trials.where(go_trials['rt'] != -1, max_go_rt)
    sorted_go = go_replacement_df.rt.sort_values(ascending = True, ignore_index=True)
    stop_failure = stop_df.loc[stop_df.rt != -1]

    p_respond = len(stop_failure)/len(stop_df)
    avg_SSD = stop_df.ssd.mean()

    nth_index = int(np.rint(p_respond*len(sorted_go))) - 1

    if nth_index < 0:
        nth_RT = sorted_go[0]
    elif nth_index >= len(sorted_go):
        nth_RT = sorted_go[-1]
    else:
        nth_RT = sorted_go[nth_index]

    SSRT = nth_RT - avg_SSD

    return SSRT

In [5]:
def preprocess_stop_data(df):
    df = df.query(f'Phase == "test"')
    df = df[['Block', 'Phase', 'trialType', 'goStim', 'correctResponse', 'ssd', 'goResp_test.keys', 'goResp_test.corr','goResp_test.rt']]
    
    df.loc[:, 'stop_acc'] = np.where(df['trialType'] == 'stop', 
                                 np.where(df['goResp_test.keys'] == 'None', 1, 0), 
                                 np.nan)

    df.loc[:, 'go_acc'] = np.where(df['trialType'] == 'go', 
                                   np.where(df['goResp_test.keys'] == df['correctResponse'], 1, 0), 
                                   np.nan)

    df.loc[:, 'stop_failure_acc'] = np.where(
        (df['trialType'] == 'stop') & (df['goResp_test.rt'].notna()),
        np.where(df['goResp_test.keys'] == df['correctResponse'], 1, 0),
        np.nan)
    
    df.rename(columns={'goResp_test.keys': 'response', 'goResp_test.corr': 'correct', 'goResp_test.rt': 'rt'}, inplace=True)
    
    return df

In [8]:
df = pd.read_csv(data_files[0])

In [11]:
df = preprocess_stop_data(df)

In [12]:
df

,Block,Phase,trialType,goStim,correctResponse,ssd,response,correct,rt,stop_acc,go_acc,stop_failure_acc
26,1.0,test,go,squareblack.png,right,0.45,right,1.0,0.586452,NaN,1.0,NaN
27,1.0,test,go,squareblack.png,right,0.45,right,1.0,0.557513,NaN,1.0,NaN
28,1.0,test,stop,circleblack.png,left,0.45,left,1.0,0.607001,0.0,NaN,1.0
29,1.0,test,stop,circleblack.png,left,0.40,None,0.0,NaN,1.0,NaN,NaN
30,1.0,test,go,circleblack.png,left,0.45,left,1.0,0.591434,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
167,3.0,test,go,squareblack.png,right,0.15,right,1.0,0.424344,NaN,1.0,NaN
168,3.0,test,go,circleblack.png,left,0.15,left,1.0,0.412977,NaN,1.0,NaN
169,3.0,test,go,circleblack.png,left,0.15,left,1.0,0.462122,NaN,1.0,NaN
170,3.0,test,stop,squareblack.png,right,0.15,None,0.0,NaN,1.0,NaN,NaN


In [19]:
for file in data_files:
    
    df = pd.read_csv(file)
    
    df = preprocess_stop_data(df)
    
    go_rt = df.query("trialType == 'go'").rt.mean()
    stop_fail_rt = df.query("trialType == 'stop'").rt.mean()
    avg_ssd = df.query("trialType == 'stop'").ssd.mean()
    
    print(f'go rt: {go_rt}')
    print(f'stop fail rt: {stop_fail_rt}')
    print(f'avg_ssd: {avg_ssd}')
    print(f'SSRT: {compute_SSRT(df)}')
    

go rt: 0.5065265443020811
stop fail rt: 0.441186660900712
avg_ssd: 0.29375
SSRT: 0.503424844890833
go rt: 0.5096488002066811
stop fail rt: 0.4202728122472763
avg_ssd: 0.32083333333333336
SSRT: 0.4906485634545485
go rt: 0.5097213212478285
stop fail rt: 0.4566815004724523
avg_ssd: 0.3083333333333333
SSRT: 0.5649401920537154
